In [9]:
gROOT->LoadMacro("../../../../RootUtils/my-atlasrootstyle/AtlasStyle.C")

(int) 0


In [10]:
SetAtlasStyle()


Applying ATLAS style settings...



In [11]:
// read the input data
auto fref = TFile::Open("OutputROOT.20200707.BL30_piplus.root");
auto tg_lp_ref = (TGraphErrors*)fref->Get("tg_lp");
auto f = TFile::Open("OutputROOT.20200721.BL30LCNNnoC2piplus.root");
auto tg_lp = (TGraphErrors*)f->Get("tg_lp");
auto tg_c1 = (TGraphErrors*)f->Get("tg_c1");
// auto tg_c2 = (TGraphErrors*)f->Get("tg_c2");
// setup the plot

In [13]:
double A[4] = { 4.0026, 20.1797, 83.7980, 131.2930 };
std::vector<TGraphErrors*> tg;
for (int i = 0; i < 4; ++i) {
  tg.push_back(new TGraphErrors(4));
  for (int j = 0; j < 4; ++j) {
    double x = 0;
    double yval = 0;
    double yerr = 0;
    x = tg_lp->GetX()[j];
    yval = tg_lp->GetY()[j];
    yval += tg_c1->GetY()[j] * std::pow(A[i], 1. / 3.);
    // yval += tg_c2->GetY()[j] * std::pow(A[i], 2. / 3.);
    yerr += std::pow(tg_lp->GetEY()[j], 2);
    yerr += std::pow(tg_c1->GetEY()[j] * std::pow(A[i], 1. / 3.), 2);
    // yerr += std::pow(tg_c2->GetEY()[j] * std::pow(A[i], 2. / 3.), 2);
    yerr += 2 * std::pow(A[i], 1./3.) * tg_lp->GetEY()[j] * tg_c1->GetEY()[j] * (-0.950);
    // yerr += 2 * std::pow(A[i], 2./3.) * tg_lp->GetEY()[j] * tg_c2->GetEY()[j] * (0.6);
    // yerr += 2 * std::pow(A[i], 1./3.) * std::pow(A[i], 2./3.) * tg_c1->GetEY()[j] * tg_c2->GetEY()[j] * (-0.950);
    yerr = std::sqrt(yerr);
    tg[i]->SetPoint(j, x, yval);
    tg[i]->SetPointError(j, 0, yerr);
//     std::cout << i << ", point " << j << ", x = " << x << ", y = " << yval << ", yerr = " << yerr <<std::endl;
  }
}

In [14]:
TF1 *f1 = new TF1("f1", "(([1]+2*([2]-[3]))/(2*[0]))*x*(log(1/(x*x))-1+x*x)/(1-x*x)", 0, 1);
f1->SetParName(0, "KAPPA");
f1->SetParName(1, "MP");
f1->SetParName(2, "NU");
f1->SetParName(3, "EPS");
f1->SetParameter(0, 1.0);
f1->SetParameter(1, 0.938);
f1->SetParameter(2, 13.1);
f1->SetParameter(3, 0.0);
// f1->FixParameter(0, 1.0);
f1->FixParameter(1, 0.938);
f1->FixParameter(2, 13.1);
// f1->FixParameter(3, 2.4);

In [16]:
tg_lp_ref->Fit("f1");
for (int i = 0; i < 4; ++i) { tg[i]->Fit("f1"); }

 FCN=23.6994 FROM MIGRAD    STATUS=CONVERGED      33 CALLS          34 TOTAL
                     EDM=3.89172e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  KAPPA        9.19315e-01   1.71580e+00   1.13373e-04  -1.89071e-02
   2  MP           9.38000e-01     fixed    
   3  NU           1.31000e+01     fixed    
   4  EPS         -7.07878e-01   2.66469e+01   1.75888e-03  -1.21743e-03
 FCN=8.53477 FROM MIGRAD    STATUS=CONVERGED      26 CALLS          27 TOTAL
                     EDM=7.82235e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  KAPPA        9.20634e-01   4.38365e+00   2.80965e-04   2.10542e-04
   2  MP           9.38000e-01     fixed    
   3  NU           1.31000e+01     fixed    

In [17]:
int colors[4] = {kAzure,kViolet,kPink,kOrange};
for (int i = 0; i < 4; ++i) {
  tg[i]->SetMarkerStyle(i + 21);
  tg[i]->SetMarkerColor(colors[i]);
  tg[i]->SetLineColor(colors[i]);
  tg[i]->SetLineWidth(2);
}

tg_lp_ref->SetTitle(";z_{h};L_{c} (fm)");
tg_lp_ref->SetLineWidth(2);
tg_lp_ref->GetYaxis()->SetRangeUser(0,15);
tg_lp_ref->GetXaxis()->SetLimits(0,1);
tg_lp_ref->GetXaxis()->SetNdivisions(505);

In [18]:
TCanvas* cnv_lc = new TCanvas("cnv_lc", "cnv_lc", 800, 600);
tg_lp_ref->Draw("APEL");
tg[0]->Draw("PLSAME");
tg[1]->Draw("PLSAME");
tg[2]->Draw("PLSAME");
tg[3]->Draw("PLSAME");
tg_lp_ref->Draw("PSAME");

TLegend* leg = new TLegend(0.2, 0.7, 0.35, 0.92);
leg->SetTextFont(42);
leg->SetTextSize(0.03);
leg->SetBorderSize(0);
leg->SetFillStyle(0);
leg->AddEntry(tg_lp_ref,"L_{c} - Baseline model","pel");
leg->AddEntry(tg[0],"L_{c}(He)","pel");
leg->AddEntry(tg[1],"L_{c}(Ne)","pel");
leg->AddEntry(tg[2],"L_{c}(Kr)","pel");
leg->AddEntry(tg[3],"L_{c}(Fe)","pel");
leg->Draw();

In [20]:
cnv_lc->SaveAs("cnv_lc.pdf")

Info in <TCanvas::Print>: pdf file cnv_lc.pdf has been created
